In [1]:
%load_ext autoreload
%autoreload 2
!sudo rm /tmp/libtpu_lockfile
import penzai
from penzai import pz
import jax.numpy as jnp
import jax_smi
import jax
jax_smi.initialise_tracking()
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
max_seq_len = 64
batch_size = 16

In [3]:
from micrlhf.llama import LlamaTransformer
from micrlhf.caching_llama import FoldedLlamaKVCachingTransformer
from micrlhf.flash import flashify

llama = LlamaTransformer.from_pretrained(("models/llama-3-70b-1.gguf", "models/llama-3-70b-2.gguf"),
                                         device_map="auto:mp=4", load_on_cpu=True,
                                        #  transpose_rotary=False
                                        #  transpose_rotary=True
                                         )
print("Folding and caching...")
llama, cache = FoldedLlamaKVCachingTransformer.from_uncached(llama, max_seq_len, {"batch": batch_size})
# llama = flashify(llama)

Folding and caching...


TypeError: can't multiply sequence by non-int of type 'jaxlib.xla_extension.PjitFunction'

In [5]:
llama = llama.to_tpu()

In [7]:
import gc
gc.collect();

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama3-ChatQA-1.5-70B")
prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
], tokenize=False)

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from micrlhf.sampling import sample
texts, cache = sample((llama, cache), tokenizer, prompt, batch_size=batch_size, max_seq_len=max_seq_len,
       do_sample=True, return_only_completion=True)
texts

StructureMismatchError: Error while checking the input of a layer of type NamedEinsum:
Mismatch while checking structures:
At root[1]: Named shape mismatch between value {'batch': 16, 'projection': 128, 'layer': 80, 'kv_heads': 8, 'seq': 64} and pattern {'seq': var('I')['tkv'], 'kv_heads': var('I')['h'], 'projection': var('I')['p'], 'batch': var('B')['batch']:=16}:
  Unexpected names in value's named shape: ['layer']
  After inlining var('B') = {'batch': 16} from root[0]

In [ ]:
next(iter(cache.kv_caches.values()))

(<NamedArray bfloat16(| batch:16, projection:128, layer:80, kv_heads:8, seq:128) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:167_772_160 (wrapping jax.Array)>,
 <NamedArray bfloat16(| batch:16, projection:128, layer:80, kv_heads:8, seq:128) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:167_772_160 (wrapping jax.Array)>)